### Setup

1. Make a copy of `../.env.example` to `../.env` and fill the `OPENAI_API_KEY` and `SDK_API_KEY` environment variables.
2. This demo assumes that chainlit-cloud is running locally.
3. Run the cells to install the dependencies and the local sdk package.

In [1]:
pip install -r ../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -e ..

Obtaining file:///Users/thibautpatel/github/chainlit-sdks/python-client
  Preparing metadata (setup.py) ... done
  Attempting uninstall: chainlit-sdk
    Found existing installation: chainlit-sdk 0.1.0
    Uninstalling chainlit-sdk-0.1.0:
      Successfully uninstalled chainlit-sdk-0.1.0
  Running setup.py develop for chainlit-sdk

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Example

In [6]:
import asyncio
import json
import uuid

import chainlit_sdk
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

sdk = chainlit_sdk.ChainlitSDK(batch_size=2)
chainlit_sdk.instrument_openai()

In [7]:
# Start a conversation

thread_id = uuid.uuid4()

welcome_message = "What's your name? "
with sdk.observer.step(type="message", thread_id=thread_id) as step:
    step.set_parameter("content", welcome_message)
    step.set_parameter("role", "assistant")

text = input(welcome_message)

What's your name?  thibaut


In [8]:
# reply with an openai completion

with sdk.observer.step(type="message", thread_id=thread_id) as step:
    step.set_parameter("content", text)
    step.set_parameter("role", "user")
    with sdk.observer.step(type="run", thread_id=thread_id) as step:
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line.",
                },
                {
                    "role": "assistant",
                    "content": welcome_message,
                },
                {
                    "role": "user",
                    "content": text,
                },
            ],
        )
        with sdk.observer.step(type="message", thread_id=thread_id) as step:
            print("")
            print(completion.choices[0].message.content)
            step.set_parameter("content", completion.choices[0].message.content)
            step.set_parameter("role", "assistant")



Thibaut,  you are a beacon of strength and resilience, capable of overcoming any obstacle that comes your way. Keep shining and pursuing your dreams!


In [9]:
sdk.wait_until_queue_empty() # wait for all msgs to be in the cloud instance

In [10]:
# Get the thread

print("\nSearching for the thread", thread_id, "...")
steps = await sdk.api.get_thread(id=str(thread_id))

thread = steps

print(json.dumps(thread, indent=2))


Searching for the thread 1bbbdddd-c440-4b8e-8b49-8628a7448a67 ...
{
  "data": {
    "thread": {
      "id": "1bbbdddd-c440-4b8e-8b49-8628a7448a67",
      "steps": [
        {
          "id": "88c1b447-7f24-4961-998e-5a92ab679479",
          "parentId": null,
          "startTime": 1696895361,
          "endTime": 1696895361,
          "type": "MESSAGE",
          "input": null,
          "output": null,
          "metadata": {},
          "tags": [],
          "operatorName": null,
          "operatorRole": null,
          "generation": null,
          "feedback": null
        },
        {
          "id": "c0953b86-9d7c-43a1-a290-a9c3334bd8cb",
          "parentId": null,
          "startTime": 1696895368,
          "endTime": 1696895370,
          "type": "LLM",
          "input": null,
          "output": null,
          "metadata": {},
          "tags": [],
          "operatorName": null,
          "operatorRole": null,
          "feedback": null,
          "generation": null
     